In [1]:
import pandas as pd
import numpy as np
from random import randint
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()
# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

#load data from memory
data=pd.read_pickle('/scratch/lnw8px/GAN/data.pkl')

In [2]:
data.head()

,user,repitition,gesture,x,y,z,crop_x,crop_y,crop_z
0,U07,5,16,"[8.580819, 8.580819, 8.887277, 10.726024, 10.7...","[3.983952, 3.830723, 3.524265, 6.895301, 6.588...","[3.217807, 3.217807, 4.903325, 10.572795, 9.50...","[8.580819, 8.580819, 8.580819, 8.580819, 8.580...","[8.580819, 8.580819, 8.580819, 8.580819, 8.580...","[8.580819, 8.580819, 8.580819, 8.580819, 8.580..."
1,U07,2,16,"[9.193734, 9.040505, 9.040505, 10.572795, 12.2...","[1.37906, 1.225831, 1.225831, 3.830723, 1.0726...","[3.830723, 3.983952, 5.822699, 12.717999, 11.7...","[9.193734, 9.193734, 9.193734, 9.193734, 9.193...","[9.193734, 9.193734, 9.193734, 9.193734, 9.193...","[9.193734, 9.193734, 9.193734, 9.193734, 9.193..."
2,U07,13,16,"[8.274362, 8.121132, 9.040505, 11.032481, 11.3...","[4.443638, 4.29041, 5.209783, 7.967903, 7.5082...","[2.75812, 2.298434, 5.516241, 11.33894, 9.3469...","[8.274362, 8.274362, 8.274362, 8.274362, 8.274...","[8.274362, 8.274362, 8.274362, 8.274362, 8.274...","[8.274362, 8.274362, 8.274362, 8.274362, 8.274..."
3,U07,14,16,"[8.121132, 8.42759, 7.661446, 9.80665, 10.2663...","[4.443638, 4.137181, 5.66947, 7.967903, 7.9679...","[2.911349, 3.064578, 5.516241, 9.193734, 9.500...","[8.121132, 8.121132, 8.121132, 8.121132, 8.121...","[8.121132, 8.121132, 8.121132, 8.121132, 8.121...","[8.121132, 8.121132, 8.121132, 8.121132, 8.121..."
4,U07,3,16,"[9.040505, 9.346964, 8.121132, 11.33894, 13.02...","[0.306458, 0.153229, -1.991976, -0.612916, -1....","[4.137181, 4.443638, 6.588843, 12.871228, 12.4...","[9.040505, 9.040505, 9.040505, 9.040505, 9.040...","[9.040505, 9.040505, 9.040505, 9.040505, 9.040...","[9.040505, 9.040505, 9.040505, 9.040505, 9.040..."


In [3]:
'''
Define GAN models
'''

'\nDefine GAN models\n'

In [4]:
def get_models(hidden1,hidden2):
    class GeneratorModel(nn.Module):
        def __init__(self):
            super(GeneratorModel, self).__init__()
            input_dim = 100 + 20
            output_dim = 30*3
            self.label_embedding = nn.Embedding(20, 20)

            self.hidden_layer1 = nn.Sequential(
                nn.Linear(input_dim, hidden1),
                nn.LeakyReLU(0.2)
            )

            self.hidden_layer2 = nn.Sequential(
                nn.Linear(hidden1, hidden2),
                nn.LeakyReLU(0.2),
                nn.Dropout(0.3)
            )

            self.hidden_layer3 = nn.Sequential(
                nn.Linear(hidden2, output_dim),
                nn.Tanh()
            )

        def forward(self, x, labels):
            c = self.label_embedding(labels)
            x = torch.cat([x,c], 1)
            output = self.hidden_layer1(x)
            output = self.hidden_layer2(output)
            output = self.hidden_layer3(output)
            output = output.view(-1,3,30)
            return output.to(device)

    class DiscriminatorModel(nn.Module):
        def __init__(self):
            super(DiscriminatorModel, self).__init__()
            input_dim = 30*3 + 20
            output_dim = 1
            self.label_embedding = nn.Embedding(20, 20)

            self.hidden_layer1 = nn.Sequential(
                nn.Linear(input_dim, hidden1),
                nn.LeakyReLU(0.2),
                nn.Dropout(0.3)
            )
            
            self.hidden_layer2 = nn.Sequential(
                nn.Linear(hidden1, hidden2),
                nn.LeakyReLU(0.2),
                nn.Dropout(0.3)
            )

            self.hidden_layer3 = nn.Sequential(
                nn.Linear(hidden2, output_dim),
                nn.Sigmoid()
            )

        def forward(self, x, labels):
            c = self.label_embedding(labels)
            x = torch.cat([x, c], 1)
            output = self.hidden_layer1(x)
            output = self.hidden_layer2(output)
            output = self.hidden_layer3(output)
            return output.to(device)

    discriminator = DiscriminatorModel()
    generator = GeneratorModel()
    discriminator.to(device)
    generator.to(device)
    
    return generator,discriminator

In [6]:
generator,discriminator=get_models(hidden1=256,hidden2=64)
generator.load_state_dict(torch.load('/scratch/lnw8px/GAN/models/generator.pt'))
generator.to(device)

GeneratorModel(
  (label_embedding): Embedding(20, 20)
  (hidden_layer1): Sequential(
    (0): Linear(in_features=120, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
  )
  (hidden_layer2): Sequential(
    (0): Linear(in_features=256, out_features=64, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Dropout(p=0.3, inplace=False)
  )
  (hidden_layer3): Sequential(
    (0): Linear(in_features=64, out_features=90, bias=True)
    (1): Tanh()
  )
)

In [7]:
print(fake_labels[15])
plt.plot(generated[15])

NameError: name 'fake_labels' is not defined

In [8]:
'''
normalize data
based on max and min values found on the dataprep section
'''

'\nnormalize data\nbased on max and min values found on the dataprep section\n'

In [9]:
def get_norm_data(array,val):
    return list(np.array(array)/val)

data['x_norm']=data['x'].apply(get_norm_data,args=[40])
data['y_norm']=data['y'].apply(get_norm_data,args=[80])
data['z_norm']=data['z'].apply(get_norm_data,args=[80])

In [10]:
'''
create onehot labels
'''

'\ncreate onehot labels\n'

In [11]:
num_classes=20
def to_onehot(num):
    onehot = np.zeros(num_classes,dtype=int)
    onehot[num-1] = 1
    return onehot

data['label']=data['gesture'].apply(to_onehot)

In [12]:
'''
crop data
'''

'\ncrop data\n'

In [13]:
def crop_array(array):
    #crop_len/10 is the length in seconds
    crop_len=30
    #pad array if its shorter than crop_len
    if(len(array)<crop_len):
        a=np.pad(array,pad_width=[0,(crop_len-len(array))],mode='edge')
    else:
        max_start=len(array)-crop_len
        start=randint(0,max_start)
        a=array[start:start+crop_len]
    return a 

crop_x=data['x_norm'].apply(crop_array)
crop_y=data['y_norm'].apply(crop_array)
crop_z=data['z_norm'].apply(crop_array)

data['crop_x']=crop_x
data['crop_y']=crop_y
data['crop_z']=crop_z

In [14]:
'''
select only the required columns
'''

'\nselect only the required columns\n'

In [15]:
data=data.drop(columns=['repitition','x','y','z','x_norm','y_norm','z_norm'])
data.head()

,user,gesture,crop_x,crop_y,crop_z,label
0,U07,16,"[0.214520475, 0.214520475, 0.22218192499999997...","[0.0497994, 0.0478840375, 0.044053312500000004...","[0.040222587500000004, 0.040222587500000004, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,U07,16,"[0.22984334999999997, 0.226012625, 0.226012625...","[0.01723825, 0.015322887499999998, 0.015322887...","[0.0478840375, 0.0497994, 0.0727837375, 0.1589...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,U07,16,"[0.20685905, 0.2030283, 0.226012625, 0.2758120...","[0.055545475, 0.053630124999999994, 0.06512228...","[0.0344765, 0.028730424999999997, 0.0689530125...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,U07,16,"[0.2030283, 0.21068975, 0.19153615, 0.24516624...","[0.055545475, 0.0517147625, 0.070868375, 0.099...","[0.0363918625, 0.038307225, 0.0689530125, 0.11...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,U07,16,"[0.226012625, 0.2336741, 0.2030283, 0.2834735,...","[0.003830725, 0.0019153625, -0.0248997, -0.007...","[0.0517147625, 0.055545475, 0.0823605375, 0.16...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [16]:
def get_max(array):
    return np.max(array)
def get_min(array):
    return np.min(array)
print(np.max(data['crop_x'].apply(get_max).values),np.min(data['crop_x'].apply(get_min).values))
print(np.max(data['crop_y'].apply(get_max).values),np.min(data['crop_y'].apply(get_min).values))
print(np.max(data['crop_z'].apply(get_max).values),np.min(data['crop_z'].apply(get_min).values))

0.8312668750000001 -0.9998186
0.9768342999999999 -0.4558559875
0.9768342999999999 -0.8121131874999999


In [17]:
'''
seperate train and test data.
one user is test. All others is train
'''

'\nseperate train and test data.\none user is test. All others is train\n'

In [18]:
users=data['user'].unique()
test_user=users[0]
train_users=users[users!=test_user]

train_data=data[data['user'].isin(train_users)]
test_data=data[data['user']==test_user]

In [19]:
train_data.head()

,user,gesture,crop_x,crop_y,crop_z,label
400,U03,11,"[0.0, 0.0, 0.068953, 0.0842759, 0.0651223, 0.0...","[0.0057460875, 0.0057460875, 0.0497994, 0.0900...","[0.12258312499999999, 0.12641385, 0.1225831249...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
401,U03,11,"[0.003830725, 0.00766145, 0.02681505, 0.076614...","[0.01723825, 0.01723825, 0.026815062499999997,...","[0.1244984875, 0.1244984875, 0.12641385, 0.134...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
402,U03,11,"[0.030645774999999997, 0.030645774999999997, 0...","[-0.0057460875, -0.0057460875, 0.045968675, 0....","[0.1283292125, 0.1244984875, 0.122583124999999...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
403,U03,11,"[0.00766145, 0.02681505, 0.061291575, 0.076614...","[0.0095768125, 0.0057460875, 0.044053312500000...","[0.1206677625, 0.1283292125, 0.118752412499999...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
404,U03,11,"[0.022984325, 0.0497994, 0.0651223, 0.0842759,...","[0.00766145, 0.0095768125, 0.05363012499999999...","[0.1244984875, 0.1244984875, 0.1130063125, 0.1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."


In [20]:
def scale(row):
    scaled=(row-np.min(row))/(np.max(row)-np.min(row))
    return scaled

def get_batch(data,batch_size):
    feature_list=[]
    label_list=[]
    batch=data.sample(n=batch_size)
    for i in range(len(batch)):
        features=[]
        features.append(batch.iloc[i]['crop_x'])
        features.append(batch.iloc[i]['crop_y'])
        features.append(batch.iloc[i]['crop_z'])
        features=np.array(features)
        feature_list.append(features)
        label_list.append(batch.iloc[i]['gesture']-1)
    features=np.array(feature_list)
    features=torch.from_numpy(features)
    features=features.to(device)
    features=features.float()
    labels=np.array(label_list)
    labels=torch.from_numpy(labels)
    labels=labels.to(device)
    labels=labels.long()
    return features,labels

def get_generated_batch(batch_size):
    noise = torch.randn(batch_size,100).to(device)
    labels = torch.randint(0, 20, (batch_size,)).to(device)
    generated=generator(noise,labels)
    
    return generated,labels

def get_comb_batch(data,real_bs,gen_bs):
    real_features,real_labels=get_batch(data=data,batch_size=real_bs)
    gen_features,gen_labels=get_generated_batch(gen_bs)
    features=torch.cat([real_features,gen_features],0)
    ar=features.detach().cpu().numpy()
    labels=torch.cat([real_labels,gen_labels],0)
    return features,labels

In [21]:
features,labels=get_comb_batch(train_data,10,12)

In [22]:
features.shape

torch.Size([22, 3, 30])

In [23]:
'''
Define the CNN
'''

'\nDefine the CNN\n'

In [24]:
def get_model(num_filters1,num_filters2): 
    
    features,labels=get_batch(train_data,batch_size=10)
    conv1 = (nn.Conv1d(in_channels=3, out_channels=num_filters1, kernel_size=2, stride=1).float()).to(device)
    pool1 = (nn.MaxPool1d(kernel_size=2).float()).to(device)
    
    conv2 = (nn.Conv1d(in_channels=num_filters1, out_channels=num_filters2, kernel_size=2, stride=1).float()).to(device)
    pool2 = (nn.MaxPool1d(kernel_size=2).float()).to(device)


    output = conv1(features)
    output=pool1(output)
    output = conv2(output)
    output=pool2(output)
    linear_in_features=output.shape[1]*output.shape[2]
    print(linear_in_features)

    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv1d(in_channels=3, out_channels=num_filters1, kernel_size=2, stride=1)
            self.pool = nn.MaxPool1d(kernel_size=2)
            self.bn1 = nn.BatchNorm1d(100)
            self.conv2 = nn.Conv1d(in_channels=num_filters1, out_channels=num_filters2, kernel_size=2, stride=1)
            self.bn2 = nn.BatchNorm1d(48)
            self.fc1 = nn.Linear(in_features=linear_in_features,out_features=20)
            self.softmax = nn.Softmax(dim=1)
            self.relu = nn.ReLU()
            self.drop = nn.Dropout(0.5)

        def forward(self, x):
            x = self.conv1(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.bn1(x)
            x = self.conv2(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.bn2(x)
            x = self.drop(x)
            x = x.view(-1,linear_in_features)
            x = self.fc1(x)
            #x = self.softmax(x)
            return x
    model=Net()
    model.to(device)
    model.train()
    return model
   

In [25]:
model=get_model(num_filters1=100,num_filters2=48)
model

288


Net(
  (conv1): Conv1d(3, 100, kernel_size=(2,), stride=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv1d(100, 48, kernel_size=(2,), stride=(1,))
  (bn2): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=288, out_features=20, bias=True)
  (softmax): Softmax(dim=1)
  (relu): ReLU()
  (drop): Dropout(p=0.5, inplace=False)
)

In [29]:
def get_test_acc_loss():
    features,labels=get_batch(test_data,batch_size=len(test_data))
    pred=model(features)
    predmax=torch.max(pred,1)[1]
    iscorrect=(predmax==labels)
    num_correct=torch.sum(iscorrect).item()
    num_total=iscorrect.shape[0]
    acc=num_correct/num_total
    loss=criterion(pred,labels).item()
    return acc,loss

In [68]:
out_file='/home/lnw8px/models/GAN/CNN_results.txt'
epochs=20
lr=0.0005
batch_size=32
real_ratio=0.5
real_bs=int(batch_size*real_ratio)
gen_bs=batch_size-real_bs
max_test_acc=0
model=get_model(num_filters1=100,num_filters2=48)
criterion = nn.CrossEntropyLoss()

for i in range(epochs):
    model.train()
    loss_list=[]
    print('lr='+str(lr))
    f = open(out_file, "a")
    f.write('lr='+str(lr)+'\n')
    f.close()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for j in range(len(train_data)):
        features,labels=get_comb_batch(train_data,real_bs=real_bs,gen_bs=gen_bs)

        model.zero_grad()
        optimizer.zero_grad()

        pred=model(features)
        loss = criterion(pred, labels)
        loss.backward()
        optimizer.step()

        loss_list.append(loss.item())
    loss_list=np.array(loss_list)
    model.eval()
    test_acc,test_loss=get_test_acc_loss()
    if(test_acc>max_test_acc):
        print('saving model')
        torch.save(model.state_dict(),'/scratch/lnw8px/GAN/models/CNN.pt')
        max_test_acc=test_acc
    print('train loss='+str(np.mean(loss_list))+' test loss='+str(test_loss))
    print('test accuracy='+str(test_acc))
    f = open(out_file, "a")
    f.write('train loss='+str(np.mean(loss_list))+' test loss='+str(test_loss)+' test accuracy='+str(test_acc)+'\n')
    f.close()
    lr=lr*0.9

288
lr=0.0005
saving model
train loss=0.256713866131718 test loss=0.5688561797142029
test accuracy=0.7875
lr=0.00045000000000000004
saving model
train loss=0.062154529231425046 test loss=0.5730488300323486
test accuracy=0.81
lr=0.00040500000000000003
saving model
train loss=0.04514386713431618 test loss=0.45203810930252075
test accuracy=0.8575
lr=0.0003645
train loss=0.03846713950219216 test loss=0.4870862066745758
test accuracy=0.84
lr=0.00032805000000000003
train loss=0.03175386962337437 test loss=0.513716459274292
test accuracy=0.8125
lr=0.000295245
train loss=0.02912063195763116 test loss=0.5126988291740417
test accuracy=0.8175
lr=0.0002657205
train loss=0.0265193657916247 test loss=0.5576711893081665
test accuracy=0.8225
lr=0.00023914845
train loss=0.024610176763485207 test loss=0.4715043008327484
test accuracy=0.84
lr=0.000215233605
train loss=0.022629656030896755 test loss=0.5573824644088745
test accuracy=0.815
lr=0.0001937102445
train loss=0.02068152740697031 test loss=0.491467

In [45]:
#model.eval()
#torch.save(model.state_dict(),'/scratch/lnw8px/GAN/models/CNN_GAN_0.7.pt')

In [80]:
model=get_model(num_filters1=100,num_filters2=48)
model.load_state_dict(torch.load('/scratch/lnw8px/GAN/models/CNN.pt'))

288


<All keys matched successfully>

In [83]:
model.eval()
get_test_acc_loss()

(0.8575, 0.4520381987094879)

In [76]:
def get_per_class_acc(test_data):
    features,labels=get_batch(test_data,batch_size=len(test_data))
    pred=model(features)
    predmax=torch.max(pred,1)[1]
    iscorrect=(predmax==labels)
    iscorrect=iscorrect.cpu().numpy()
    predmax=predmax.cpu().numpy()
    ones=np.ones(len(iscorrect))
    res=np.bincount(predmax,iscorrect)
    counts=np.bincount(predmax,ones)
    perclass_acc=res/counts
    return perclass_acc

In [54]:
get_per_class_acc(test_data)

array([0.95238095, 0.90909091, 0.71428571, 0.95238095, 1.        ,
       1.        , 1.        , 0.81818182, 0.63157895, 0.90909091,
       1.        , 0.95      , 0.95238095, 1.        , 0.74074074,
       0.94117647, 1.        , 0.86363636, 0.86956522, 1.        ])

(20,)